There is some differences between Jean's and Zhenxia's alignments. I want to examine the differences closely. I am going to focus on:

* DRR014573
* SRR3086588
* SRR942869

In [1]:
import pandas as pd
pd.options.display.max_columns=999

In [26]:
df = pd.read_csv('../../data/jean/Mapping_QC/ZhenXia.Data_Summary.ZhenXia.txt', sep='\t', skiprows=2, index_col=0)
df.loc[['DRR014573', 'SRR3086588', 'SRR942869']]

,Unnamed: 1,Total reads,Z Total reads,% Total reads,% Z Total reads,% difference,Mapped,Z Mapped,% Mapped,% Z Mapped,% difference.1,Mapped uniquely,Z Mapped uniquely,% Mapped uniquely,% Z Mapped uniquely,% difference.2,Mapped non uniquely,Z Mapped non_unique,% Mapped non uniquely,% Z non_uniquely,% difference.3,Unmapped,Z unmapped_reads,% Unmapped,% Z unmapped_reads,% difference.4,Intron support: a read supporting 2 introns counts 2,Z spliced read,% Intron supports: a read supporting 2 intron counts 2,% Z spliced read,% difference.5,Reads mapped in transcripts,"Z Read fragments mapped in genes, a read touching 2 exons counts 2",% Reads mapped in transcripts,"% Z Read fragments mapped in genes, a read touching 2 exons counts 2",% difference.6,Stranding,Z Stranding,% difference.7,Significantly expressed genes,Touched Genes,Z expressed genes : is it touched or significant ?,Difference,Sex,Z sex,Stage,Z dev_stage,Tissue-System,Z tissue,Z cell_type,Z sample_type,Stage summary,Z genotype,Magic sample title,Sequencing length
### Run,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DRR014573,NaN,91947.0,91947.0,100.0,100.0,0.0,91069.0,296.0,99.05,0.32,-98.72,89415.0,242.0,97.25,0.26,-96.98,1654.0,54.0,1.8,0.06,-1.74,878.0,91651.0,0.95,99.68,98.72,0.0,102.0,0.00,0.11,0.11,65371.0,219.0,71.10,0.24,-70.86,52.16,50.45,1.72,412,2201,110,2091,NaN,unknown,L3,larva,Body_part; Other; salivary gland and fat body ...,whole_organism,mix,whole_organism,salivary gland and fat body dissected from 20 ...,Twenty 3rd instar larvae of D. melanogaster (C...,Penicillium-fungus uninfected 3rd instar larva...,294.0
SRR3086588,NaN,29560480.0,29560480.0,100.0,100.0,0.0,28723030.0,3385.0,97.17,0.01,-97.16,28722320.0,426.0,97.16,0.00,-97.16,710.0,2959.0,0.0,0.01,0.01,837450.0,29557090.0,2.83,99.99,97.16,117.0,9.0,0.00,0.00,-0.00,28720688.0,289.0,97.16,0.00,-97.16,86.28,77.97,8.31,1,22,153,-131,Male,male,Adult,adult,Germline_and_development; testis,testis,mix,whole_tissue,NaN,HMS00184,"Model organism or animal sample from, tudormt ...",50.0
SRR942869,NaN,144718704.0,144718704.0,100.0,100.0,0.0,129969800.0,12657088.0,89.81,8.75,-81.06,128809800.0,524808.0,89.01,0.36,-88.64,1160000.0,12132280.0,0.8,8.38,7.58,14748904.0,132061600.0,10.19,91.25,81.06,708082.0,327149.0,0.49,0.23,-0.26,128550736.0,190612.0,88.83,0.13,-88.70,99.88,47.34,52.54,2379,5520,6266,-746,NaN,unknown,Embryo,embryo,NaN,whole_organism,mix,whole_organism,NaN,yw,"footprint 0-2 hour embryo replicate A, whole 0...",57.0


We can see from this table that while the number of total reads is the same between Jean and Zhenxia, the number of mapped reads is completely different, with Zhenxia failing to map a large portion of reads.

# Prep Temp Folder and References

In [70]:
%%bash
hisat2 --version |head -n1 || conda install hisat2 -y
(printf "cutadapt " && cutadapt --version) || conda install cutadapt -y
samtools --version | head -n1 || conda install samtools -y

/Users/fearjm/opt/miniconda3/bin/hisat2-align-s version 2.0.4
cutadapt 1.9.1
samtools 1.3.1


In [66]:
%%bash --err error
REF=~/tmp/ncbi_remap/references
if [ ! -e $REF ]; then
    mkdir -p $REF
fi

rsync -av bio:/data/Oliverlab/data/FlyBase/FB2015_04/dmel_ERCC* $REF/

receiving file list ... done

sent 20 bytes  received 302 bytes  49.54 bytes/sec
total size is 457850364  speedup is 1421895.54


# Run Hisat2

First I want to look at using Hisat2 to align one of the above samples.

In [82]:
%%bash
cd ~/tmp/ncbi_remap

hisat2 -p 4 \
    -x references/dmel_ERCC \
    --dta --sra-acc DRR014573 \
    --known-splicesite-infile references/dmel_ERCC.splicesites.txt \
    -S DRR014573.hisat.sra.sam

91947 reads; of these:
  91947 (100.00%) were unpaired; of these:
    61151 (66.51%) aligned 0 times
    16819 (18.29%) aligned exactly 1 time
    13977 (15.20%) aligned >1 times
33.49% overall alignment rate


In [83]:
%%bash
source activate hisat2
cd ~/tmp/ncbi_remap

bam_stat.py -i DRR014573.hisat.sra.sam

discarding /Users/fearjm/opt/miniconda3/bin from PATH
prepending /Users/fearjm/opt/miniconda3/envs/hisat2/bin to PATH
Load SAM file ...  Done

#==================================================
#All numbers are READ count
#==================================================

Total records:                          97105

QC failed:                              0
Optical/PCR duplicate:                  0
Non primary hits                        5158
Unmapped reads:                         61151
mapq < mapq_cut (non-unique):           3659

mapq >= mapq_cut (unique):              27137
Read-1:                                 0
Read-2:                                 0
Reads map to '+':                       12766
Reads map to '-':                       14371
Non-splice reads:                       17758
Splice reads:                           9379
Reads mapped in proper pairs:           0
Proper-paired reads map to different chrom:0


As you can see, bam_stat reports strange counts. But, either way my alignment percentage is low (~18%). So next I am going to try to dump the SRA to a FASTQ and look at the FASTQ.

In [77]:
%%bash 
cd ~/tmp/ncbi_remap

fastq-dump --split-spot -M 35 DRR014573

Rejected 91947 READS because READLEN < 35
Read 91947 spots for DRR014573
Written 91947 spots for DRR014573


In [84]:
%%bash
cd ~/tmp/ncbi_remap

hisat2 -p 4 \
    -x references/dmel_ERCC \
    --dta -U DRR014573.fastq \
    --known-splicesite-infile references/dmel_ERCC.splicesites.txt \
    -S DRR014573.hisat.fq.sam

91947 reads; of these:
  91947 (100.00%) were unpaired; of these:
    61150 (66.51%) aligned 0 times
    16813 (18.29%) aligned exactly 1 time
    13984 (15.21%) aligned >1 times
33.49% overall alignment rate


In [87]:
%%bash
cd ~/tmp/ncbi_remap

bowtie2 -p 4 -x ~/references/genomes/Dmelanogaster/dm6/bowtie2/dm6-noextra \
    -U DRR014573.fastq > DRR014573.bt2.sam

91947 reads; of these:
  91947 (100.00%) were unpaired; of these:
    35778 (38.91%) aligned 0 times
    29482 (32.06%) aligned exactly 1 time
    26687 (29.02%) aligned >1 times
61.09% overall alignment rate


In [88]:
%%bash
cd ~/tmp/ncbi_remap

bowtie2 -p 4 -x ~/references/genomes/Dmelanogaster/dm6/bowtie2/dm6-noextra \
    --local \
    -U DRR014573.fastq > DRR014573.bt2.sam

91947 reads; of these:
  91947 (100.00%) were unpaired; of these:
    825 (0.90%) aligned 0 times
    36591 (39.80%) aligned exactly 1 time
    54531 (59.31%) aligned >1 times
99.10% overall alignment rate


Just dumping the file using fastq-dump and aligning with bowtie2 we get better alignments, but still not close to Jean's alignments. 

In [49]:
%%bash
cd ~/tmp/ncbi_remap

cutadapt -a AGATCGGAAGAGC -a "A{50}" -a "T{50}" --trim-n -o DRR014573_2.trim.fastq DRR014573_2.fastq

This is cutadapt 1.9.1 with Python 3.5.1
Command line parameters: -a AGATCGGAAGAGC -a A{50} -a T{50} --trim-n -o DRR014573_2.trim.fastq DRR014573_2.fastq
Trimming 3 adapters with at most 10.0% errors in single-end mode ...
Finished in 4.66 s (51 us/read; 1.18 M reads/minute).

=== Summary ===

Total reads processed:                  91,947
Reads with adapters:                        81 (0.1%)
Reads written (passing filters):        91,947 (100.0%)

Total basepairs processed:    26,703,586 bp
Total written (filtered):     26,417,431 bp (98.9%)

=== Adapter 1 ===

Sequence: AGATCGGAAGAGC; Type: regular 3'; Length: 13; Trimmed: 37 times.

No. of allowed errors:
0-9 bp: 0; 10-13 bp: 1

Bases preceding removed adapters:
  A: 10.8%
  C: 45.9%
  G: 24.3%
  T: 16.2%
  none/other: 2.7%

Overview of removed sequences
length	count	expect	max.err	error counts
3	12	1436.7	0	12
5	1	89.8	0	1
6	6	22.4	0	6
9	1	0.4	0	0 1
47	1	0.0	1	0 1
74	1	0.0	1	0 1
84	1	0.0	1	0 1
104	1	0.0	1	0 1
147	1	0.0	1	0 1
161	1	